In [1]:
import nltk
nltk.download('popular')
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

## Loading DataSet


In [2]:
data = pd.read_csv("dataset.csv")
data.head(10)

,Unnamed: 0,source_text,plagiarized_text,label
0,0,Researchers have discovered a new species of b...,Scientists have found a previously unknown but...,1
1,1,The moon orbits the Earth in approximately 27....,Our natural satellite takes around 27.3 days t...,1
2,2,Water is composed of two hydrogen atoms and on...,H2O consists of 2 hydrogen atoms and 1 oxygen ...,1
3,3,The history of Rome dates back to 753 BC.,Rome has a long history that can be traced bac...,1
4,4,Pluto was once considered the ninth planet in ...,"In the past, Pluto was classified as the ninth...",1
5,5,This is a unique and original sentence.,This sentence is unique and original.,0
6,6,Artificial intelligence is reshaping industries.,AI is changing the landscape of various sectors.,0
7,7,Python is a popular programming language for d...,Data science often relies on Python as a widel...,0
8,8,The Earth revolves around the Sun in a nearly ...,Our planet follows an almost circular path as ...,0
9,9,Paris is the capital of France.,France's capital city is Paris.,0


In [3]:
data['label'].value_counts()

,count
label,
0,187
1,183


In [4]:
data.shape

(370, 4)

In [8]:
def preprocess_text(text):
  # removing punctuation
  text = text.translate(str.maketrans(" ", " ", string.punctuation))
  # converting to lowercase()
  text = text.lower()
  # removing stopwrods
  stop_words = set(stopwords.words('english'))
  text = " ".join(word for word in text.split() if word not in stop_words)
  return text

proprocess_text("This is my %#$#$!?/ text use for dummy text")


'text use dummy text'

## Cleaning Data

In [9]:
data['source_text'] = data['source_text'].apply(preprocess_text)
data['plagiarized_text'] = data['plagiarized_text'].apply(preprocess_text)

In [10]:
data

,Unnamed: 0,source_text,plagiarized_text,label
0,0,researchers discovered new species butterfly a...,scientists found previously unknown butterfly ...,1
1,1,moon orbits earth approximately 273 days,natural satellite takes around 273 days comple...,1
2,2,water composed two hydrogen atoms one oxygen atom,h2o consists 2 hydrogen atoms 1 oxygen atom,1
3,3,history rome dates back 753 bc,rome long history traced back 753 bc,1
4,4,pluto considered ninth planet solar system,past pluto classified ninth planet suns planet...,1
...,...,...,...,...
365,397,playing musical instruments enhances creativity,creativity enhanced playing musical instruments,0
366,398,studying history helps understanding present,understanding present aided studying history,0
367,399,listening classical music improve focus,focus improved listening classical music,0
368,400,practicing yoga enhances physical flexibility,physical flexibility enhanced practicing yoga,0


## Vectorization


In [14]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data["source_text"] + " " + data["plagiarized_text"])

In [15]:
y = data["label"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42 )

## Logistic Regression

In [20]:
model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("Accuracy ", accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("Confusion ", confusion_matrix(y_test,y_pred))


Accuracy  0.8243243243243243
Classification                precision    recall  f1-score   support

           0       0.79      0.86      0.82        35
           1       0.86      0.79      0.83        39

    accuracy                           0.82        74
   macro avg       0.83      0.83      0.82        74
weighted avg       0.83      0.82      0.82        74

Confusion  [[30  5]
 [ 8 31]]


## Random Forest


In [22]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators= 100, random_state = 42)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("Accuracy ", accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("Confusion ", confusion_matrix(y_test,y_pred))

Accuracy  0.7972972972972973
Classification                precision    recall  f1-score   support

           0       0.71      0.97      0.82        35
           1       0.96      0.64      0.77        39

    accuracy                           0.80        74
   macro avg       0.83      0.81      0.79        74
weighted avg       0.84      0.80      0.79        74

Confusion  [[34  1]
 [14 25]]


## Naive Baye's

In [23]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)
y_pred= model.predict(X_test)

print("Accuracy ", accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("Confusion ", confusion_matrix(y_test,y_pred))

Accuracy  0.8648648648648649
Classification                precision    recall  f1-score   support

           0       0.86      0.86      0.86        35
           1       0.87      0.87      0.87        39

    accuracy                           0.86        74
   macro avg       0.86      0.86      0.86        74
weighted avg       0.86      0.86      0.86        74

Confusion  [[30  5]
 [ 5 34]]


## SVM

In [24]:
from sklearn.svm import SVC

model = SVC(kernel = 'linear', random_state = 42)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("Accuracy ", accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("Confusion ", confusion_matrix(y_test,y_pred))

Accuracy  0.8783783783783784
Classification                precision    recall  f1-score   support

           0       0.86      0.89      0.87        35
           1       0.89      0.87      0.88        39

    accuracy                           0.88        74
   macro avg       0.88      0.88      0.88        74
weighted avg       0.88      0.88      0.88        74

Confusion  [[31  4]
 [ 5 34]]


In [25]:
import pickle
pickle.dump(model, open('model.pkl','wb'))
pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl','wb'))

In [26]:
model = pickle.load(open('model.pkl', 'rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))

In [27]:
def detect(input_text):
    vectorized_text = tfidf_vectorizer.transform([input_text])
    result = model.predict(vectorized_text)
    return "Plagiarim Detected" if result[0] == 1 else "No Plagiarism"

In [28]:
input_text = 'Researchers have discovered a new species of butterfly in the Amazon rainforest.'
detect(input_text)

'Plagiarim Detected'

In [30]:
input_text = 'Playing musical instruments enhances creativity.'
detect(input_text)

'No Plagiarism'

In [29]:
input_text = 'Practicing yoga enhances physical flexibility.'
detect(input_text)

'No Plagiarism'

In [32]:
import sklearn
sklearn.__version__

'1.6.1'